In [1]:
import numpy as np
import random
import math
import json
import os
from datetime import datetime
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D # Import từ File 4

# ---- TÁI LẬP SEED ----
random.seed(42)
np.random.seed(42)

# ==========================
# TÍNH TOÁN CƠ BẢN (Lấy phiên bản tối ưu từ File 4)
# ==========================
def compute_vs(p1, p2, v_f, v_AUV):
    """Tính vận tốc tổng hợp v_s giữa 2 vị trí p1, p2"""
    diff = p2 - p1
    L_mag = np.linalg.norm(diff)
    if L_mag == 0:
        return v_AUV

    cos_beta = np.clip(diff[2] / L_mag, -1, 1)
    beta = math.acos(cos_beta)

    # Bảo vệ chia cho 0
    cos_beta_abs = max(abs(cos_beta), 1e-9)

    inner = np.clip((v_f * cos_beta) / v_AUV, -1, 1)
    angle = beta + math.acos(inner)
    v_s = abs(math.cos(angle) * v_AUV / cos_beta_abs)
    return max(v_s, 1e-9) # Đảm bảo v_s > 0


def travel_time(path, coords, v_f, v_AUV):
    """Tính tổng thời gian di chuyển theo path (Từ File 4)"""
    total_time = 0.0
    for i in range(len(path)):
        p1 = coords[path[i]]
        p2 = coords[path[(i + 1) % len(path)]] # Quay về 0
        d = np.linalg.norm(p2 - p1)
        v_s = compute_vs(p1, p2, v_f, v_AUV)
        total_time += d / v_s
    return total_time

# ==========================
# LOGIC PHÂN CỤM (Từ File 1)
# ==========================
def calculate_objective_function(nodes, labels, centers):
    numerator = 0
    for i in range(2):
        cluster_nodes = nodes[labels == i]
        if len(cluster_nodes) > 0:
            distances = np.linalg.norm(cluster_nodes - centers[i], axis=1)
            numerator += np.sum(distances)
    denominator = np.linalg.norm(centers[0] - centers[1])
    if denominator == 0:
        return float('inf')
    return numerator / denominator

def check_subgroup_threshold(nodes, r_sen):
    if len(nodes) <= 1:
        return True
    max_distance = 0
    for i in range(len(nodes)):
        for j in range(i + 1, len(nodes)):
            dist = np.linalg.norm(nodes[i] - nodes[j])
            max_distance = max(max_distance, dist)
    return max_distance <= r_sen

def kmeans_with_best_T(nodes, N=30):
    best_T = float('inf')
    best_labels = None
    best_centers = None
    for _ in range(N):
        kmeans = KMeans(n_clusters=2, n_init=1)
        labels = kmeans.fit_predict(nodes)
        centers = kmeans.cluster_centers_
        T = calculate_objective_function(nodes, labels, centers)
        if T < best_T:
            best_T = T
            best_labels = labels.copy()
            best_centers = centers.copy()
    return best_labels, best_centers, best_T

def cluster_split(nodes, node_ids, node_data=None, r_sen=50, R=20, N=30, max_depth=10, depth=0):
    size_ok = len(nodes) <= R
    distance_ok = check_subgroup_threshold(nodes, r_sen)
    
    if (size_ok and distance_ok) or depth >= max_depth:
        center = np.mean(nodes, axis=0)
        return [{"node_ids": node_ids, "nodes": nodes, "center": center, "node_data": node_data if node_data else {}}]
    
    labels, centers, best_T = kmeans_with_best_T(nodes, N)
    
    clusters = []
    for i in range(2):
        sub_nodes = nodes[labels == i]
        sub_ids = [node_ids[j] for j in range(len(node_ids)) if labels[j] == i]
        sub_node_data = {}
        if node_data:
            for node_id in sub_ids:
                if node_id in node_data:
                    sub_node_data[node_id] = node_data[node_id]
        clusters += cluster_split(sub_nodes, sub_ids, sub_node_data, r_sen, R, N, max_depth, depth + 1)
    return clusters

# ==========================
# LOGIC CHỌN CH (Từ File 1)
# ==========================
def choose_cluster_head(cluster, node_data_dict):
    node_ids = cluster["node_ids"]
    center = np.array(cluster["center"])
    if len(node_ids) == 0:
        return None
    
    best_node = None
    min_distance = float('inf')
    max_energy_at_min_dist = -1
    
    for node_id in node_ids:
        if node_id not in node_data_dict:
            continue
        node_idx = node_ids.index(node_id)
        node_pos = cluster["nodes"][node_idx]
        distance = np.linalg.norm(np.array(node_pos) - center)
        energy = node_data_dict[node_id].get('residual_energy', 0.0)
        
        if distance < min_distance or (distance == min_distance and energy > max_energy_at_min_dist):
            min_distance = distance
            max_energy_at_min_dist = energy
            best_node = node_id
    
    return best_node if best_node is not None else node_ids[0]

# ==========================
# LOGIC NĂNG LƯỢNG & MẠNG (Từ File 1)
# ==========================
def compute_energy(best_time, n_members):
    """
    Tính năng lượng tiêu thụ ĐỘNG (dynamic) của File 1
    """
    G, L = 100, 1024
    P_t, P_r, P_idle, DR, DR_i = 1.6e-3, 0.8e-3, 0.1e-3, 4000, 1e6

    # Năng lượng cho Member Node
    E_tx_MN = G * P_t * L / DR
    E_idle_MN = (best_time - G * L / DR) * P_idle
    E_total_MN = E_tx_MN + E_idle_MN

    # Năng lượng cho Cluster Head
    E_rx_TN = G * P_r * L * n_members / DR
    E_tx_TN = G * P_t * L * n_members / DR_i
    E_idle_TN = (best_time - (G*L*n_members/DR) - (G*L*n_members/DR_i)) * P_idle
    E_total_TN = E_rx_TN + E_tx_TN + E_idle_TN

    return {"Member": {"E_total": E_total_MN}, "Target": {"E_total": E_total_TN}}

def update_energy(all_nodes, clusters, best_time):
    """Cập nhật năng lượng dựa trên mô hình động của File 1."""
    for cid, cinfo in clusters.items():
        ch = cinfo.get('cluster_head')
        nodes = cinfo.get('nodes', [])
        n_members = len([n for n in nodes if n != ch])
        
        energy_report = compute_energy(best_time, n_members)
        
        for nid in nodes:
            if nid not in all_nodes: continue
            if nid == ch:
                all_nodes[nid]['residual_energy'] -= energy_report['Target']['E_total']
            else:
                all_nodes[nid]['residual_energy'] -= energy_report['Member']['E_total']
            all_nodes[nid]['residual_energy'] = max(all_nodes[nid]['residual_energy'], 0.0)

def remove_dead_nodes(all_nodes, clusters):
    dead = [nid for nid, info in list(all_nodes.items()) if info['residual_energy'] <= 0]
    for nid in dead:
        del all_nodes[nid]
    new_clusters = {}
    for cid, cinfo in clusters.items():
        alive_nodes = [nid for nid in cinfo.get('nodes', []) if nid in all_nodes]
        if alive_nodes:
            new_c = dict(cinfo)
            new_c['nodes'] = alive_nodes
            new_clusters[cid] = new_c
    return new_clusters, dead

def recluster(all_nodes, node_positions, r_sen=50, R=20):
    ids = sorted(list(all_nodes.keys()))
    if len(ids) == 0:
        return {}
    coords = np.array([node_positions[nid] for nid in ids])
    raw_clusters = cluster_split(coords, ids, all_nodes, r_sen=r_sen, R=R)
    clusters = {}
    for i, c in enumerate(raw_clusters):
        center = c['center'].tolist()
        node_ids = c['node_ids']
        ch = choose_cluster_head(c, all_nodes)
        clusters[i] = {'nodes': node_ids, 'center': center, 'cluster_head': ch}
    return clusters

# ==========================
# HÀM TÌM ĐƯỜNG PSO (Từ File 4)
# ==========================
def get_swap_sequence(A, B):
    seq = []
    temp = A.copy()
    for i in range(1, len(A)): # Bắt đầu từ 1 vì 0 là cố định
        if temp[i] != B[i]:
            # Tìm vị trí của phần tử B[i] trong temp
            try:
                j = temp.index(B[i])
                seq.append((i, j))
                temp[i], temp[j] = temp[j], temp[i]
            except ValueError:
                # Xảy ra khi A và B không có cùng các phần tử (không nên xảy ra)
                pass 
    return seq

def apply_velocity(position, velocity):
    pos = position.copy()
    for i, j in velocity:
        if i > 0 and j > 0 and i < len(pos) and j < len(pos): # Bảo vệ index
            pos[i], pos[j] = pos[j], pos[i]
    return pos

def pso_tsp_3d_time(coords, v_f=1.0, v_AUV=3.0,
                    n_particles=40, max_iter=200,
                    w=0.7, c1=0.5, c2=0.5, init_gbest=None, verbose=True):
    
    n_cities = len(coords)
    if n_cities < 2: return [0], 0.0
    if n_cities == 2: return [0, 1], travel_time([0, 1], coords, v_f, v_AUV)
    
    cities = list(range(1, n_cities))

    # Khởi tạo quần thể
    swarm = [[0] + random.sample(cities, len(cities)) for _ in range(n_particles)]
    velocities = [[] for _ in range(n_particles)]

    # Đánh giá ban đầu
    costs = [travel_time(p, coords, v_f, v_AUV) for p in swarm]
    pbest = [p.copy() for p in swarm]
    pbest_cost = costs.copy()

    # Xác định gbest
    if init_gbest is not None and len(init_gbest) == n_cities:
        gbest = init_gbest.copy()
        gbest_cost = travel_time(gbest, coords, v_f, v_AUV)
    else:
        best_idx = int(np.argmin(pbest_cost))
        gbest = pbest[best_idx].copy()
        gbest_cost = pbest_cost[best_idx]

    # --- Vòng lặp chính ---
    for t in range(max_iter):
        inertia = 0.7 - 0.5 * (t / max_iter) # Giảm dần inertia

        for i in range(n_particles):
            xi, vi = swarm[i], velocities[i]

            # Giữ lại 1 phần vận tốc cũ
            v_new = vi[:int(inertia * len(vi))]

            # Ảnh hưởng cá nhân (pbest)
            if random.random() < c1:
                seq_pb = get_swap_sequence(xi, pbest[i])
                if seq_pb:
                    v_new += random.sample(seq_pb, min(2, len(seq_pb))) # Thêm 2 swap ngẫu nhiên

            # Ảnh hưởng toàn cục (gbest)
            if random.random() < c2:
                seq_gb = get_swap_sequence(xi, gbest)
                if seq_gb:
                    v_new += random.sample(seq_gb, min(2, len(seq_gb))) # Thêm 2 swap ngẫu nhiên

            # Cập nhật vị trí và vận tốc
            new_x = apply_velocity(xi, v_new)
            new_cost = travel_time(new_x, coords, v_f, v_AUV)
            
            swarm[i], velocities[i] = new_x, v_new

            # Cập nhật pbest và gbest
            if new_cost < pbest_cost[i]:
                pbest[i], pbest_cost[i] = new_x, new_cost
                if new_cost < gbest_cost:
                    gbest, gbest_cost = new_x, new_cost

        if verbose and t % 50 == 0:
            print(f"    [PSO Iter {t:3d}]: Best time = {gbest_cost:.4f}")

    if verbose:
         print(f"    [PSO Iter {max_iter:3d}]: Final Best time = {gbest_cost:.4f}")
    return gbest, gbest_cost

def multi_pso_tsp(coords, v_f=1.2, v_AUV=3.0, n_outer=5, verbose=True, **kwargs):
    """Chạy PSO nhiều lần để cải thiện hội tụ (Từ File 4)"""
    prev_gbest, prev_cost = None, float('inf')

    for outer in range(1, n_outer + 1):
        if verbose:
            print(f"   [PSO Outer loop {outer}/{n_outer}]")
            
        gbest, cost = pso_tsp_3d_time(coords, v_f=v_f, v_AUV=v_AUV,
                                      init_gbest=prev_gbest, verbose=verbose, **kwargs)
        
        if cost < prev_cost:
            prev_gbest, prev_cost = gbest, cost
        
        # Có thể thêm điều kiện dừng sớm nếu không cải thiện
        # if abs(cost - prev_cost) < 1e-6:
        #     print("   → Converged! Stop early.")
        #     break

    return prev_gbest, prev_cost

# ==========================
# HÀM MAIN (Từ File 1, đã chỉnh sửa)
# ==========================
def main():
    """
    Hàm chính mô phỏng (cấu trúc từ File 1).
    ĐÃ THAY THẾ 'nearest_neighbor_path' BẰNG 'multi_pso_tsp'.
    """
    input_dir = "/kaggle/input/test2-input"
    output_dir = "result_PSO_simulation" # Đổi tên thư mục output
    os.makedirs(output_dir, exist_ok=True)

    if not os.path.exists(input_dir):
        print(f"❌ Lỗi: Thư mục {input_dir} không tồn tại!")
        return
    files = [f for f in os.listdir(input_dir) if f.endswith('.json')]
    if len(files) == 0:
        print(f"❌ Không tìm thấy file dữ liệu nào trong {input_dir}")
        return

    INITIAL_ENERGY = 100.0
    v_f = 1.2
    v_AUV = 3.0
    results_summary = []

    for filename in files:
        input_path = os.path.join(input_dir, filename)
        print(f"\n{'='*60}")
        print(f"=== Đang xử lý file: {filename} ===")
        print(f"{'='*60}")
        
        try:
            with open(input_path, 'r') as f:
                data = json.load(f)
        except Exception as e:
            print(f"❌ Lỗi đọc file {filename}: {e}")
            continue

        node_positions = {}
        all_nodes = {}
        
        # Logic đọc dữ liệu từ File 1 (đọc danh sách node)
        if isinstance(data, list):
            print("📄 Định dạng: Danh sách nodes")
            for node in data:
                nid = node['id']
                all_nodes[nid] = {
                    'initial_energy': node.get('initial_energy', INITIAL_ENERGY),
                    'residual_energy': node.get('residual_energy', INITIAL_ENERGY)
                }
                node_positions[nid] = (node['x'], node['y'], node['z'])
        else:
            print(f"❌ Cấu trúc file {filename} không được hỗ trợ (cần list of nodes)")
            continue

        total_nodes = len(all_nodes)
        print(f"📊 Tổng số node ban đầu: {total_nodes}")

        cycle = 0
        alive_log = []
        energy_log = []

        # Vòng lặp mô phỏng
        while True:
            cycle += 1
            alive_log.append(len(all_nodes))
            total_energy = sum(all_nodes[n]['residual_energy'] for n in all_nodes)
            energy_log.append(total_energy)

            alive_ratio = len(all_nodes)/total_nodes if total_nodes > 0 else 0
            print(f"\n--- Cycle {cycle} --- | Alive: {alive_ratio*100:.2f}% ({len(all_nodes)}/{total_nodes}) | Energy: {total_energy:.2f}J")

            if alive_ratio < 0.9:
                print("🛑 Dừng mô phỏng vì < 90% node còn sống")
                break

            # Phân cụm lại (từ File 1)
            clusters = recluster(all_nodes, node_positions)
            if len(clusters) == 0:
                print("🛑 Không còn node -> Dừng")
                break

            # Tạo danh sách tọa độ các tâm cụm để cho PSO
            sorted_keys = sorted(clusters.keys())
            centers = [(0, 0, 0)]  # Base station là điểm 0
            for k in sorted_keys:
                centers.append(tuple(clusters[k]['center']))
            
            center_coords = np.array(centers)
            
            # --- LOG THÊM VÀO (1) ---
            print(f"   Tìm đường đi (PSO) cho {len(center_coords)} điểm (1 BS + {len(clusters)} cụm).")
            # ------------------------

            # --- THAY THẾ TÌM ĐƯỜNG ---
            # Bỏ: path_indices = nearest_neighbor_path(centers)
            # Bỏ: best_time = travel_time(path_indices, centers, v_f, v_AUV)
            
            # Gọi PSO để tìm đường (từ File 4)
            # Giảm số lượng để mô phỏng chạy nhanh
            best_path_indices, best_time = multi_pso_tsp(
                center_coords, v_f, v_AUV,
                n_outer=3,          # Giảm số vòng lặp ngoài
                n_particles=30,     # Giảm số hạt
                max_iter=80,        # Giảm số vòng lặp trong
                verbose=False       # Tắt log của PSO để đỡ nhiễu
            )
            # --- KẾT THÚC THAY THẾ ---

            # --- LOG THÊM VÀO (2) ---
            print(f"   Pathfinding (PSO): Best time = {best_time:.2f}s")
            # ------------------------

            # Cập nhật năng lượng (từ File 1)
            update_energy(all_nodes, clusters, best_time)
            clusters, dead_nodes = remove_dead_nodes(all_nodes, clusters)
            
            if dead_nodes:
                print(f"   ⚡ {len(dead_nodes)} node(s) đã hết năng lượng")

        # Lưu kết quả (từ File 1)
        meta = {
            'input_file': filename,
            'initial_total_nodes': total_nodes,
            'cycles_completed': cycle - 1,
            'final_alive_nodes': len(all_nodes),
            'final_alive_ratio': len(all_nodes)/total_nodes if total_nodes > 0 else 0
        }
        
        output_json = os.path.join(output_dir, f"result_{filename}")
        with open(output_json, "w") as f:
            json.dump(meta, f, indent=4)

        results_summary.append((filename, cycle - 1))
        print(f"\n✅ File {filename}: {cycle - 1} cycles hoàn thành")

        # Plot: Alive nodes per cycle (từ File 1)
        plt.figure(figsize=(10, 6))
        plt.plot(range(len(alive_log)), alive_log, marker='o', linewidth=2)
        plt.title(f"Số node sống theo chu kỳ - {filename}", fontsize=14, fontweight='bold')
        plt.xlabel("Chu kỳ", fontsize=12)
        plt.ylabel("Nodes alive", fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"alive_nodes_{filename}.png"), dpi=150)
        plt.close()

        # Plot: Total energy per cycle (từ File 1)
        plt.figure(figsize=(10, 6))
        plt.plot(range(len(energy_log)), energy_log, marker='s', linewidth=2, color='orange')
        plt.title(f"Năng lượng toàn mạng theo chu kỳ - {filename}", fontsize=14, fontweight='bold')
        plt.xlabel("Chu kỳ", fontsize=12)
        plt.ylabel("Total energy (J)", fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"energy_{filename}.png"), dpi=150)
        plt.close()

    # Summary plot (từ File 1)
    if results_summary:
        labels = [x[0] for x in results_summary]
        values = [x[1] for x in results_summary]
        plt.figure(figsize=(10, 6))
        plt.plot(labels, values, marker='o', linewidth=2, markersize=8)
        plt.title("AUV cycles completed per dataset (PSO Path)", fontsize=14, fontweight='bold')
        plt.xticks(rotation=45, ha='right')
        plt.ylabel("Cycles", fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "summary_cycles.png"), dpi=150)
        plt.close()
        
        print(f"\n{'='*60}")
        print(f"✅ Hoàn thành! Kết quả đã lưu tại: {output_dir}")
        print(f"{'='*60}")
    else:
        print("\n⚠️  Không có kết quả nào được tạo ra")

if __name__ == '__main__':
    main()


=== Đang xử lý file: nodes_200.json ===
📄 Định dạng: Danh sách nodes
📊 Tổng số node ban đầu: 200

--- Cycle 1 --- | Alive: 100.00% (200/200) | Energy: 20000.00J
   Tìm đường đi (PSO) cho 30 điểm (1 BS + 29 cụm).
   Pathfinding (PSO): Best time = 159.69s

--- Cycle 2 --- | Alive: 100.00% (200/200) | Energy: 19987.15J
   Tìm đường đi (PSO) cho 32 điểm (1 BS + 31 cụm).
   Pathfinding (PSO): Best time = 177.10s

--- Cycle 3 --- | Alive: 100.00% (200/200) | Energy: 19974.06J
   Tìm đường đi (PSO) cho 31 điểm (1 BS + 30 cụm).
   Pathfinding (PSO): Best time = 250.44s

--- Cycle 4 --- | Alive: 100.00% (200/200) | Energy: 19959.45J
   Tìm đường đi (PSO) cho 32 điểm (1 BS + 31 cụm).
   Pathfinding (PSO): Best time = 192.32s

--- Cycle 5 --- | Alive: 100.00% (200/200) | Energy: 19946.06J
   Tìm đường đi (PSO) cho 31 điểm (1 BS + 30 cụm).
   Pathfinding (PSO): Best time = 198.46s

--- Cycle 6 --- | Alive: 100.00% (200/200) | Energy: 19932.49J
   Tìm đường đi (PSO) cho 34 điểm (1 BS + 33 cụm).
  